## 물건의 수요량 예측
댓글 5만개를 읽어서 다음주의 수요량을 예측 하는 모델을 만든다.
LSTM모델을 이용할 것입니다.

몇개가 팔릴 것인지를 예측해야 한다.

### 데이터 수집
https://smartstore.naver.com/i/v1/reviews/paged-reviews?page=4&pageSize=20&merchantNo=500071344&originProductNo=280423930&sortType=REVIEW_CREATE_DATE_DESC

네이버 쇼핑 검색 결과에서 특정 물건의 모든 댓글 정보를 모두 수집 합니다.
한페이지에 30개씩 1692번 콜하면 모든 댓글 수집이 가능 합니다.

In [46]:
import pandas as pd
from datetime import datetime

# df = pd.read_json('r.json', dtype={'createDate':'date'})
df = pd.read_json('r.json')
df.index = df['id']
df['createDate'] = pd.to_datetime(df['createDate'], format="%Y-%m-%d")
# df['createDate']=df['createDate'].apply(lambda _ : datetime.strptime(_,'%Y%m%d'))

In [47]:
df.iloc[:, 0:30]

,id,reviewServiceType,reviewType,reviewContentClassType,reviewContent,createDate,reviewDisplayStatusType,repurchase,reviewScore,reviewRankingScore,...,productNo,productName,productOptionContent,largeCategorizeCategoryId,middleCategorizeCategoryId,smallCategorizeCategoryId,detailCategorizeCategoryId,productUrl,originProductNo,eventTitle
id,,,,,,,,,,,,,,,,,,,,,
752808213,752808213,SELLBLOG,AFTER_USE,TEXT,잘쓰고있습니다 좋네요 다음에 또 구매하겠습니다.,2019-11-07 10:06:46+00:00,NORMAL,1.0,5,-78.117270,...,280423930,체어스코 네이처 컴퓨터의자 책상의자,제품명: 네이처 스탠다드 / 색상: 라임오렌지,50000004,50000105,50001240,50003299,https://smartstore.naver.com/main/products/280...,280423930,
752801857,752801857,SELLBLOG,NORMAL,PHOTO,감사합니다~~~~~,2019-11-07 09:53:50+00:00,NORMAL,NaN,5,-78.087470,...,280423930,체어스코 네이처 컴퓨터의자 책상의자,제품명: 네이처 베이직 / 색상: 노블레스블랙,50000004,50000105,50001240,50003299,https://smartstore.naver.com/main/products/280...,280423930,
752796423,752796423,SELLBLOG,NORMAL,PHOTO,배송 빠르고 상품. 가격 모두 만족합니다,2019-11-07 09:42:38+00:00,NORMAL,NaN,5,-78.000680,...,280423930,체어스코 네이처 컴퓨터의자 책상의자,제품명: 네이처 베이직 / 색상: 노블레스블랙,50000004,50000105,50001240,50003299,https://smartstore.naver.com/main/products/280...,280423930,
752790791,752790791,SELLBLOG,AFTER_USE,TEXT,아주 좋와요~~~^^,2019-11-07 09:30:48+00:00,NORMAL,NaN,5,-78.236910,...,280423930,체어스코 네이처 컴퓨터의자 책상의자,제품명: 네이처 리얼라텍스 / 색상: 파이어와인,50000004,50000105,50001240,50003299,https://smartstore.naver.com/main/products/280...,280423930,
752788406,752788406,SELLBLOG,NORMAL,PHOTO,방석이 없으면 몸이 흘러내려가는게 단점이라면 단점이지만 그 외에는 편하고 사무일하기...,2019-11-07 09:25:51+00:00,NORMAL,NaN,4,-77.969154,...,280423930,체어스코 네이처 컴퓨터의자 책상의자,제품명: 네이처 리얼라텍스 / 색상: 인텔그레이,50000004,50000105,50001240,50003299,https://smartstore.naver.com/main/products/280...,280423930,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879823199,879823199,SELLBLOG,NORMAL,TEXT,가성비가 좋아요 다음에 산다면 옵션을 추가할꺼같아요,2020-06-20 02:38:29+00:00,NORMAL,NaN,5,-76.436905,...,280423930,체어스코 의자 네이처 책상의자 컴퓨터의자,제품선택: 블랙바디 기본 세트 / 색상선택: 옐로우그린,50000004,50000105,50001240,50003299,https://smartstore.naver.com/main/products/280...,280423930,
879788303,879788303,SELLBLOG,NORMAL,TEXT,주문제작임에도 빠른배송감사합니다,2020-06-20 01:33:35+00:00,NORMAL,NaN,5,-76.509500,...,280423930,체어스코 의자 네이처 책상의자 컴퓨터의자,제품선택: 블랙바디 허리받침 세트 / 색상선택: 메탈그레이,50000004,50000105,50001240,50003299,https://smartstore.naver.com/main/products/280...,280423930,
879768831,879768831,SELLBLOG,NORMAL,TEXT,전반적으로 마음에 들어요,2020-06-20 00:53:30+00:00,NORMAL,NaN,4,-76.527270,...,280423930,체어스코 의자 네이처 책상의자 컴퓨터의자,제품선택: 블랙바디 라텍스+허리받침 세트 / 색상선택: 퓨어블랙,50000004,50000105,50001240,50003299,https://smartstore.naver.com/main/products/280...,280423930,


In [51]:
pd.to_datetime(df['createDate'], format="%Y-%m-%d").strftime('%Y-%m')

AttributeError: 'Series' object has no attribute 'strftime'

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50750 entries, 752808213 to 879728447
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype              
---  ------                          --------------  -----              
 0   id                              50750 non-null  int64              
 1   reviewServiceType               50750 non-null  object             
 2   reviewType                      50750 non-null  object             
 3   reviewContentClassType          50750 non-null  object             
 4   reviewContent                   50750 non-null  object             
 5   createDate                      50750 non-null  datetime64[ns, UTC]
 6   reviewDisplayStatusType         50750 non-null  object             
 7   repurchase                      11429 non-null  float64            
 8   reviewScore                     50750 non-null  int64              
 9   reviewRankingScore              50738 non-null  float64            
 10

In [5]:
df.describe()
df.columns

Index(['id', 'reviewServiceType', 'reviewType', 'reviewContentClassType',
       'reviewContent', 'createDate', 'reviewDisplayStatusType', 'repurchase',
       'reviewScore', 'reviewRankingScore', 'writerMemberId',
       'writerMemberIdNo', 'writerMemberNo', 'writerMemberProfileImageUrl',
       'checkoutMerchantNo', 'checkoutMerchantId', 'channelId',
       'channelServiceType', 'orderNo', 'productOrderNo', 'productNo',
       'productName', 'productOptionContent', 'largeCategorizeCategoryId',
       'middleCategorizeCategoryId', 'smallCategorizeCategoryId',
       'detailCategorizeCategoryId', 'productUrl', 'originProductNo',
       'eventTitle', 'reviewEvaluationValueSeqs', 'writerMemberMaskedId',
       'profileImageSourceType', 'reviewAttaches', 'repThumbnailAttach',
       'repThumbnailTagNameDescription', 'merchantNo', 'merchantName',
       'isMyReview', 'reviewTopics', 'parentReviewSeq', 'helpCount',
       'modifyDate', 'reviewCommentIds', 'reviewComments',
       'reviewIns

,id,repurchase,reviewScore,reviewRankingScore,writerMemberNo,checkoutMerchantNo,channelId,productNo,largeCategorizeCategoryId,middleCategorizeCategoryId,smallCategorizeCategoryId,detailCategorizeCategoryId,originProductNo,isMyReview,parentReviewSeq,helpCount,bestReview
createDate,,,,,,,,,,,,,,,,,
2015-03-13 00:00:00+00:00,1.402860e+08,NaN,5.000000,-90.684944,0.0,0.0,500071344.0,280423930.0,50000004.0,50000105.0,50001239.0,50003683.0,280423930.0,0.0,NaN,NaN,NaN
2015-03-14 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-15 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-16 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-17 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-11 00:00:00+00:00,3.151962e+09,0.081081,4.729730,-74.263347,0.0,0.0,500071344.0,280423930.0,50000004.0,50000105.0,50001240.0,50003299.0,280423930.0,0.0,3.129197e+09,NaN,NaN
2021-04-12 00:00:00+00:00,3.152580e+09,0.020408,4.591837,-74.255329,0.0,0.0,500071344.0,280423930.0,50000004.0,50000105.0,50001240.0,50003299.0,280423930.0,0.0,3.114076e+09,NaN,NaN
2021-04-13 00:00:00+00:00,3.153281e+09,0.038462,4.673077,-74.217018,0.0,0.0,500071344.0,280423930.0,50000004.0,50000105.0,50001240.0,50003299.0,280423930.0,0.0,3.118817e+09,2.0,NaN


### 데이터 전처리
주로 나누어서 그 7일 동안 상품별로 몇개의 리뷰가 달렸는지부터


reviewScore:1~5

productOptionContent:옵션



In [50]:
# type(df['createDate'])


id
752808213   2019-11-07 10:06:46+00:00
752801857   2019-11-07 09:53:50+00:00
752796423   2019-11-07 09:42:38+00:00
752790791   2019-11-07 09:30:48+00:00
752788406   2019-11-07 09:25:51+00:00
                       ...           
879823199   2020-06-20 02:38:29+00:00
879788303   2020-06-20 01:33:35+00:00
879768831   2020-06-20 00:53:30+00:00
879757645   2020-06-20 00:28:27+00:00
879728447   2020-06-19 23:06:02+00:00
Name: createDate, Length: 50750, dtype: datetime64[ns, UTC]

### 정규화

In [55]:
df.set_index('createDate').groupby(pd.Grouper(freq='D')).count()


,id,reviewServiceType,reviewType,reviewContentClassType,reviewContent,reviewDisplayStatusType,repurchase,reviewScore,reviewRankingScore,writerMemberId,...,isMyReview,reviewTopics,parentReviewSeq,helpCount,modifyDate,reviewCommentIds,reviewComments,reviewInspectionPolicyReason,bestReview,bestReviewSelectDate
createDate,,,,,,,,,,,,,,,,,,,,,
2015-03-13 00:00:00+00:00,1,1,1,1,1,1,0,1,1,1,...,1,0,0,0,0,0,0,0,0,0
2015-03-14 00:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-03-15 00:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-03-16 00:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2015-03-17 00:00:00+00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-11 00:00:00+00:00,37,37,37,37,37,37,37,37,37,37,...,37,14,1,0,0,0,0,0,0,0
2021-04-12 00:00:00+00:00,49,49,49,49,49,49,49,49,49,49,...,49,16,6,0,0,0,0,0,0,0
2021-04-13 00:00:00+00:00,52,52,52,52,52,52,52,52,52,52,...,52,21,8,3,0,0,0,0,0,0
